In [ ]:
import pandas as pd
import numpy as np

In [ ]:
PATHS = [
    '../input/cicids2017/MachineLearningCSV/MachineLearningCVE/Monday-WorkingHours.pcap_ISCX.csv',
    '../input/cicids2017/MachineLearningCSV/MachineLearningCVE/Tuesday-WorkingHours.pcap_ISCX.csv',
    '../input/cicids2017/MachineLearningCSV/MachineLearningCVE/Wednesday-workingHours.pcap_ISCX.csv',
    '../input/cicids2017/MachineLearningCSV/MachineLearningCVE/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv',
    '../input/cicids2017/MachineLearningCSV/MachineLearningCVE/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv',
    '../input/cicids2017/MachineLearningCSV/MachineLearningCVE/Friday-WorkingHours-Morning.pcap_ISCX.csv',
    '../input/cicids2017/MachineLearningCSV/MachineLearningCVE/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv',
    '../input/cicids2017/MachineLearningCSV/MachineLearningCVE/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv'
    
]
df = pd.read_csv(PATHS[0])
for i in range(1,len(PATHS)):
    temp = pd.read_csv(PATHS[i])
    df = pd.concat([df,temp])

In [ ]:
# df = pd.get_dummies(df)

In [ ]:
m = df.loc[df[' Flow Packets/s'] != np.inf,' Flow Packets/s'].max()
df[' Flow Packets/s'].replace(np.inf,m,inplace=True)
m = df.loc[df['Flow Bytes/s'] != np.inf,'Flow Bytes/s'].max()
df['Flow Bytes/s'].replace(np.inf,m,inplace=True)

In [ ]:
dtypes = df.dtypes
print(f"Number of columns with Int {len(dtypes[dtypes == int])}")
print(f"Number of columns with float {len(dtypes[dtypes == float])}")
print(f"Number of columns with object {len(dtypes[dtypes == object])}")

In [ ]:
null_values = df.isna().sum()
null_values[null_values >0]

In [ ]:
null_index = np.where(df['Flow Bytes/s'].isnull())[0]
df.dropna(inplace = True)

In [ ]:
# labels = df[' Label'].unique()
# label_enc  = dict()
# for i in range(len(labels)):
#     label_enc[labels[i]] = i
# df[' Label'] = df[' Label'].map(label_enc)
# import plotly.express as px
# fig = px.imshow(df.corr())
# fig.show()


In [ ]:
# import plotly.express as px
# fig = px.imshow(df.corr())
# fig.show()

In [ ]:
df[' Destination Port']

In [ ]:
temp = df[df[' Label'] == 'BENIGN']
temp[' Destination Port'].describe()
temp = temp.sample(frac = 0.1)

In [ ]:
df = df[df[' Label'] != 'BENIGN']
df = pd.concat([df,temp])

In [ ]:
def sample_class(x,benign_sample,malicious_sample):
    res = 0
    print(x.shape)
    try:
        x = x.reset_index(drop = True)
        check = x[' Label'][0]
        if check == 'BENIGN':
            res = x.sample(benign_sample)
        else:
            res = x.sample(malicious_sample)
    except:
        pass
    return res

In [ ]:
t = df.groupby(' Label').apply(lambda x:  sample_class(x,benign_sample = 10000,malicious_sample = 5000) if x.shape[0]>5000 else x ) # sample_class(x,benign_sample = 10000,malicious_sample = 5000) if x.shape[0]>5000 else

In [ ]:
t = t.reset_index(drop = True)
t.to_csv("CICIDS2017_w_o_oversample.csv",index = False)

In [ ]:
t = pd.get_dummies(t)

In [ ]:
col = [' Destination Port', ' Flow Duration', ' Total Fwd Packets',
       ' Total Backward Packets', 'Total Length of Fwd Packets',
       ' Total Length of Bwd Packets', ' Fwd Packet Length Max',
       ' Fwd Packet Length Min', ' Fwd Packet Length Mean',
       ' Fwd Packet Length Std', 'Bwd Packet Length Max',
       ' Bwd Packet Length Min', ' Bwd Packet Length Mean',
       ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s',
       ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min',
       'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max',
       ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std',
       ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags', ' Bwd PSH Flags',
       ' Fwd URG Flags', ' Bwd URG Flags', ' Fwd Header Length',
       ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s',
       ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean',
       ' Packet Length Std', ' Packet Length Variance', 'FIN Flag Count',
       ' SYN Flag Count', ' RST Flag Count', ' PSH Flag Count',
       ' ACK Flag Count', ' URG Flag Count', ' CWE Flag Count',
       ' ECE Flag Count', ' Down/Up Ratio', ' Average Packet Size',
       ' Avg Fwd Segment Size', ' Avg Bwd Segment Size',
       ' Fwd Header Length.1', 'Fwd Avg Bytes/Bulk', ' Fwd Avg Packets/Bulk',
       ' Fwd Avg Bulk Rate', ' Bwd Avg Bytes/Bulk', ' Bwd Avg Packets/Bulk',
       'Bwd Avg Bulk Rate', 'Subflow Fwd Packets', ' Subflow Fwd Bytes',
       ' Subflow Bwd Packets', ' Subflow Bwd Bytes', 'Init_Win_bytes_forward',
       ' Init_Win_bytes_backward', ' act_data_pkt_fwd',
       ' min_seg_size_forward', 'Active Mean', ' Active Std', ' Active Max',
       ' Active Min', 'Idle Mean', ' Idle Std', ' Idle Max', ' Idle Min']

In [ ]:
train_df= df[df['folds'] != 5]
valid_df = df[df['folds'] == 5]

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
train_df[col] = scaler.fit_transform(train_df[col])
valid_df[col] = scaler.transform(valid_df[col])

# Model 

In [ ]:

def create_ae_mlp(num_columns, num_labels, hidden_units, dropout_rates, ls = 1e-2, lr = 1e-3):
    
    inp = tf.keras.layers.Input(shape = (num_columns, ))
    x0 = tf.keras.layers.BatchNormalization()(inp)
    
    #Encoder
    encoder = tf.keras.layers.GaussianNoise(dropout_rates[0])(x0)
    encoder = tf.keras.layers.Dense(hidden_units[0])(encoder)
    encoder = tf.keras.layers.BatchNormalization()(encoder)
    encoder = tf.keras.layers.Activation('swish')(encoder)
    
    #Decoder
    decoder = tf.keras.layers.Dropout(dropout_rates[1])(encoder)
    decoder = tf.keras.layers.Dense(num_columns, name = 'decoder')(decoder)
    
    
    x_ae = tf.keras.layers.Dense(hidden_units[1])(decoder)
    x_ae = tf.keras.layers.BatchNormalization()(x_ae)
    x_ae = tf.keras.layers.Activation('swish')(x_ae)
    x_ae = tf.keras.layers.Dropout(dropout_rates[2])(x_ae)

    out_ae = tf.keras.layers.Dense(num_labels, activation = 'sigmoid', name = 'ae_action')(x_ae)
    
    #Multi Layer perceptron
    x = tf.keras.layers.Concatenate()([x0, encoder])
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(dropout_rates[3])(x)
    
    for i in range(2, len(hidden_units)):
        x = tf.keras.layers.Dense(hidden_units[i])(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation('swish')(x)
        x = tf.keras.layers.Dropout(dropout_rates[i + 2])(x)
        
    out = tf.keras.layers.Dense(num_labels, activation = 'sigmoid', name = 'action')(x)
    
    model = tf.keras.models.Model(inputs = inp, outputs = [out_ae, out])
    model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = lr),
                  loss = {
                          'ae_action': tf.keras.losses.BinaryCrossentropy(label_smoothing = ls),
                          'action': tf.keras.losses.BinaryCrossentropy(label_smoothing = ls), 
                         },
                  metrics = { 
                             'ae_action': tf.keras.metrics.AUC(name = 'AUC'), 
                             'action': tf.keras.metrics.AUC(name = 'AUC'), 
                            }, 
                 )
    
    return model

In [ ]:
train_df.iloc[:,-15:].columns

In [ ]:
params = {'num_columns': len(col), 
          'num_labels': 15, 
          'hidden_units': [96, 96, 896, 448, 448, 256], 
          'dropout_rates': [0.03527936123679956, 0.038424974585075086, 0.42409238408801436, 0.10431484318345882, 0.49230389137187497, 0.32024444956111164, 0.2716856145683449, 0.4379233941604448], 
          'ls': 0, 
          'lr':1e-3, 
         }

In [ ]:
import tensorflow as tf
tf.random.set_seed(42)
import tensorflow.keras.backend as K
import tensorflow.keras.layers as layers
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

In [ ]:
batch_size = 64

In [ ]:
epochs = 10

In [ ]:
fold = 5
ckp_path = f'JSModel_{fold}.hdf5'
model = create_ae_mlp(**params)
ckp = ModelCheckpoint(ckp_path, monitor = 'val_action_AUC', verbose = 0, 
                      save_best_only = True, save_weights_only = True, mode = 'max')
es = EarlyStopping(monitor = 'val_action_AUC', min_delta = 1e-4, patience = 20, mode = 'max', 
                   baseline = None, restore_best_weights = True, verbose = 0)
history = model.fit(train_df[col], [train_df.iloc[:,-15:],train_df.iloc[:,-15:]], 
                    validation_data = (valid_df[col], [ valid_df.iloc[:,-15:],valid_df.iloc[:,-15:]]), 
                    epochs = epochs, batch_size = batch_size, callbacks = [ckp, es], verbose = True)
hist = pd.DataFrame(history.history)


In [ ]:
from sklearn.metrics import precision_recall_fscore_support,confusion_matrix
from sklearn.metrics import roc_auc_score

In [ ]:
res = model.predict(valid_df[col])
res = np.argmax(res[1],axis =1)

truth = np.argmax(valid_df.iloc[:,-15:].values,axis = 1)

truth = np.argmax(valid_df.iloc[:,-15:].values,axis = 1)
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(truth, res)
precision,recall,_,_ = precision_recall_fscore_support(truth,res)

In [ ]:
columns

In [ ]:
roc_auc_score(valid_df.iloc[:,-15:], model.predict(valid_df[col])[1], multi_class='ovr')

In [ ]:
df_cm = confusion_matrix(truth, res)
df_cm = pd.DataFrame(df_cm, columns = columns,index = columns)

In [ ]:
columns = valid_df.columns[-15:]

In [ ]:
import seaborn as sn
plot = sn.heatmap(df_cm, annot=True,cmap='Blues')

In [ ]:
plot.savefig("output.png")

In [ ]:
import plotly.graph_objects as go
import json
!pip install -U kaleido
import kaleido

In [ ]:
hist['epochs'] = [ i+1 for i in range(epochs)]

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
                            x=hist['epochs'], y=hist['val_loss'],
                            mode='lines+markers',
                            name = "Validation Loss"
                        )
            )

fig.add_trace(go.Scatter(
                            x=hist['epochs'], y=hist['loss'],
                            mode='lines+markers',
                            name = "Train Loss"
                        )
            )
fig.update_layout(title="Loss Versus Epochs for Autoencoder-MLP",
                   xaxis_title='Epoch',
                   yaxis_title='Loss')
fig.write_image("ae_loss.jpeg")
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
                            x=hist['epochs'], y=hist['val_AUC'],
                            mode='lines+markers',
                            name = "Validation AUC"
                        )
            )

fig.add_trace(go.Scatter(
                            x=hist['epochs'], y=hist['AUC'],
                            mode='lines+markers',
                            name = "Train AUC"
                        )
            )
fig.update_layout(title="AUC Versus Epochs for Autoencoder-MLP",
                   xaxis_title='Epoch',
                   yaxis_title='AUC')
fig.write_image("ae_auc.jpeg")
fig.show()